In [19]:
# Import dependencies
import subprocess
import json
from dotenv import load_dotenv
import os 
from pprint import pprint 

In [20]:
from web3 import Account, middleware, Web3
from web3.gas_strategies.time_based import medium_gas_price_strategy
from web3.middleware import geth_poa_middleware

In [21]:
# Load and set environment variables
load_dotenv()
mnemonic= os.getenv("mnemonic")

In [22]:
# Import constants.py and necessary functions from bit and web3
from constants import *
 

In [23]:
 # Create a function called `derive_wallets`
def derive_wallets(coin=BTC, mnemonic=mnemonic, depth=3):
     #./derive --key=xprv9zbB6Xchu2zRkf6jSEnH9vuy7tpBuq2njDRr9efSGBXSYr1QtN8QHRur28QLQvKRqFThCxopdS1UD61a5q6jGyuJPGLDV9XfYHQto72DAE8 --cols=path,address --coin=ZEC --numderive=3 -g
    command =f'php ./derive -g --mnemonic="{mnemonic}" --cols=all --coin={coin} --numderive={depth} --format=json'
    p = subprocess.Popen(command, stdout=subprocess.PIPE, shell=True)
    output, err = p.communicate()
    p_status = p.wait()
    return json.loads(output)

In [24]:
pprint(derive_wallets(coin=ETH))

[{'address': '0xEda32073000B5B23A3193a3b12112d96992e219D',
  'index': 0,
  'path': "m/44'/60'/0'/0/0",
  'privkey': '0x8bdbaa57f08c9d9f585c878930f7220f9bc020813b9b94ed3ac83d53aafe5e02',
  'pubkey': '023a9366ea45044f93124d3f58b073cb233440cc532f229a940472c2cf84c8ca95',
  'pubkeyhash': 'c82553693552fb813cb0023ab93ba5cbda86f5a0',
  'xprv': 'xprvA3CF4oEpGPLju8VrpTL78TqmhxBmzNxJRFZ2288eS9KYBdUgq4gpbzcpUwufRjDgpLK77TCdw98NErSfzxvVouH7DG5vJgQW2bagRLeqQX9',
  'xpub': 'xpub6GBbUJmi6ku37caKvUs7VbnWFz2GPqg9nUUcpWYFzUrX4RoqNc159nwJLCKdd4J1o18Tv1Mus2zxYP7PoLv32RhtB8mZDs9D8KRFcsLwGWc'},
 {'address': '0x0eB56C9cB3B3bafab8aD86B623A4b83827C58dd5',
  'index': 1,
  'path': "m/44'/60'/0'/0/1",
  'privkey': '0x6db833fbb825cba102858b5a47e42f4efd97e846e2e8fa73ebd6c6ec13341d9f',
  'pubkey': '029982a6704f061fb230ad067fcf60b3faf60df986fad6d4c846871e30a89c33b5',
  'pubkeyhash': '03db0ac9ed56999b614870680e41d984c27c933a',
  'xprv': 'xprvA3CF4oEpGPLjxDJ3vp45MKkY283vvFHdFS2XiiSYzmLoaSxwmgPg7nyGWsm8YbA9AXa1YfquszEnVH

In [25]:
# Create a dictionary object called coins to store the output from `derive_wallets`.
coins = {
    ETH: derive_wallets(coin=ETH),
    BTCTEST: derive_wallets(coin=BTCTEST)
}

In [26]:
# Create a function called `priv_key_to_account` that converts privkey strings to account objects.
def priv_key_to_account(coin, priv_key):
    if coin == 'BTCTEST':
        return PrivateKeyTestnet(priv_key)
    elif coin == 'ETH':
        return Account.privateKeyToAccount(priv_key)

In [27]:
# Create a function called `create_tx` that creates an unsigned transaction appropriate metadata.
def create_tx(coin, account, to, amount):
    if coin == ETH:
        gasEstimate = w3.eth.estimateGas({"from": account.address, "to": to, "value": amount})
        return{"from": account.address, "to": to, "value": amount, "gasPrice": w3.eth.gasPrice, "gas": gasEstimate, "nonce": w3.eth.getTransactionCount(account.address), "chainID": w3.eth.chain_id}
    if coin == BTCTEST:
        return PrivateKeyTestnet.prepare_transaction(account.address, [(to, amount, BTC)])

In [28]:
# Create a function called `send_tx` that calls `create_tx`, signs and sends the transaction.
def send_tx(coin, account, to, amount):
    if coin == ETH:
        raw_tx = create_tx(coin, account, to, amount)
        signed = account.signTransaction(raw_tx)
        return w3.eth.sendRawTransaction(signed.rawTransaction)
    if coin == BTCTEST:
        raw_tx = create_tx(coin, account, to, amount)
        signed = account.sign_transaction(raw_tx)
        return NetworkAPI.broadcast_tx_testnet(signed)